In [84]:
import pandas as pd
import numpy as np
import datetime
from pypfopt.efficient_frontier import EfficientFrontier
import empyrical

# Importing Data

In [85]:
TRAIN_START = datetime.datetime(2017,12,31)
TRAIN_END = TEST_START = datetime.datetime(2019,12,29)
TEST_END = datetime.datetime(2020,12,27)

Read data for CC used in this project. 
The CC have data as from Jan 2018 to Feb 2021.
There is a total of 68 CC. 

Data is resampled for each CC: The daily data is transformed into weekly data. For the market cap and price, the last value of the day in a week is taken, for the market cap, the mean value for the week is taken and for the volume, the sum of all the volume of each day is taken.

In [86]:
listOfCC = pd.read_csv("Symbols.csv")

ALL_DATA = pd.DataFrame()

for ticker in listOfCC['symbol']:
    
    filename = "CoinGecko_Data/" + ticker + '-usd-max.csv'
    temp = pd.read_csv(filename, parse_dates=['snapped_at'])
    temp = temp.rename(columns = {"snapped_at":"date"})
    temp['date'] = pd.to_datetime(temp['date']).dt.tz_localize(None)
    temp = temp.sort_values('date').set_index('date')
    temp = temp.truncate(before = TRAIN_START,after = TEST_END)
    
    temp1 = pd.to_numeric(temp['price'], downcast="float")
    temp2 = pd.to_numeric(temp['market_cap'], downcast="float")
    temp3 = pd.to_numeric(temp['total_volume'], downcast="float")
        
    temp1 = temp1.resample('W').last()
    temp2 = temp2.resample('W').mean()
    temp3 = temp3.resample('W').sum()
    temp4 = pd.DataFrame(index = [temp1.index])
    temp4 = pd.concat([temp1, temp2, temp3], axis=1)
    
    temp4 = temp4.fillna(axis = 0, method ='bfill')
    temp4['Returns'] = temp4['price'].pct_change()
    temp4['Ticker'] = ticker    
    temp4 = temp4.iloc[1:].copy()    
    ALL_DATA = pd.concat([ALL_DATA,temp4])

ALL_DATA

,price,market_cap,total_volume,Returns,Ticker
date,,,,,
2018-01-07,1.429257,5.478657e+08,200383264.0,0.468908,zrx
2018-01-14,2.090971,1.082455e+09,505867104.0,0.462978,zrx
2018-01-21,1.742747,8.778975e+08,215137936.0,-0.166537,zrx
2018-01-28,1.979192,9.018653e+08,225045584.0,0.135673,zrx
2018-02-04,1.376865,8.179149e+08,142141824.0,-0.304330,zrx
...,...,...,...,...,...
2020-11-29,14.644183,1.307519e+08,164073328.0,0.709512,zen
2020-12-06,12.489144,1.420835e+08,112744968.0,-0.147160,zen
2020-12-13,10.172314,1.121735e+08,65833872.0,-0.185508,zen


In [87]:
ALL_DATA.reset_index(drop=False, inplace=True)
dateData = ALL_DATA.set_index(['date', 'Ticker'])
dateData.sort_index(inplace=True)
dateData

price    market_cap  total_volume   Returns
date       Ticker                                                 
2018-01-07 ada      1.052854  2.605216e+10  2.750787e+09  0.371556
           adx      3.287920  1.825394e+08  1.972941e+08  0.098724
           ae       2.737842  4.608177e+08  2.592159e+07  0.942597
           ant      8.023694  1.965826e+08  2.687493e+07  0.959480
           ardr     1.817910  1.751584e+09  1.332934e+08  0.050602
...                      ...           ...           ...       ...
2020-12-27 xrp      0.295383  1.793648e+10  7.558532e+10 -0.489978
           xvg      0.007130  1.138970e+08  2.135772e+07 -0.070242
           zec     66.474121  7.047802e+08  3.573732e+09 -0.098113
           zen     11.915156  1.114705e+08  8.049540e+07 -0.013300
           zrx      0.353898  2.756317e+08  4.010995e+08 -0.164775

[10608 rows x 4 columns]

In [88]:
tickerData = ALL_DATA.set_index(['Ticker', 'date'])
tickerData.sort_index(inplace=True)
tickerData

price    market_cap  total_volume   Returns
Ticker date                                                      
ada    2018-01-07  1.052854  2.605216e+10  2.750787e+09  0.371556
       2018-01-14  0.841890  2.257185e+10  1.857563e+09 -0.200374
       2018-01-21  0.629033  1.781683e+10  1.034086e+09 -0.252832
       2018-01-28  0.630576  1.587981e+10  5.482682e+08  0.002453
       2018-02-04  0.448043  1.274673e+10  6.180009e+08 -0.289471
...                     ...           ...           ...       ...
zrx    2020-11-29  0.415663  3.149009e+08  1.041728e+09  0.003351
       2020-12-06  0.408543  3.085792e+08  3.243042e+08 -0.017129
       2020-12-13  0.392484  2.005443e+08  2.680847e+08 -0.039307
       2020-12-20  0.423716  2.881971e+08  3.250699e+08  0.079574
       2020-12-27  0.353898  2.756317e+08  4.010995e+08 -0.164775

[10608 rows x 4 columns]

In [89]:
date_list = (tickerData.loc['btc']).index
print(date_list[[104,155]])

DatetimeIndex(['2020-01-05', '2020-12-27'], dtype='datetime64[ns]', name='date', freq=None)


In [90]:
def getLongOnlyWeights(returns):
    mean = returns.mean()
    cov = returns.cov()
    ef1 = EfficientFrontier(mean,cov,weight_bounds=(0, 1))
    LongOnly_weights = ef1.max_sharpe(risk_free_rate= 0.0)
    LongOnly_weights = ef1.clean_weights() 
    LongOnly_weights = pd.DataFrame(LongOnly_weights, index=['weight'], columns=LongOnly_weights.keys())
    
    return LongOnly_weights

In [91]:
def getLongAndShortWeights(returns):
    mean = returns.mean()
    cov = returns.cov()
    ef2 = EfficientFrontier(mean,cov,weight_bounds=(-1, 1))
    LongAndShort_weights = ef2.max_sharpe(risk_free_rate= 0.0)
    LongAndShort_weights = ef2.clean_weights()    
    LongAndShort_weights = pd.DataFrame(LongAndShort_weights, index=['weight'], columns=LongAndShort_weights.keys())
    
    return LongAndShort_weights

In [92]:
marketcap_LongOnly_returns = pd.DataFrame(index= date_list[104 : 156], columns = ['marketcap_LongOnly_returns'])

for i in range(104,156,1):
    this_week = date_list[i]
    last_week = date_list[i-1]
    first_week = date_list[i-52]
    
    this_week_data = (dateData.loc[this_week]).copy()
    last_week_data = (dateData.loc[last_week]).copy()
    
    marketcap_list = ((last_week_data.sort_values(by=['market_cap'], ascending=False)).index)[:10]
    
    marketcap_data = pd.DataFrame(index= date_list[i-52 : i], columns = list(marketcap_list))
    
    for ticker in marketcap_list:
        marketcap_data[ticker] = tickerData.Returns.loc[ticker].loc[first_week:this_week]
        
    weights = getLongOnlyWeights(marketcap_data)
    totalReturns = 0
    
    for ticker in marketcap_list:
        totalReturns += weights[ticker]['weight'] * this_week_data.Returns.loc[ticker]
        
    marketcap_LongOnly_returns.loc[this_week,'marketcap_LongOnly_returns'] = totalReturns

In [93]:
liquid_LongOnly_returns = pd.DataFrame(index= date_list[104 : 156], columns = ['liquid_LongOnly_returns'])

for i in range(104,156,1):
    this_week = date_list[i]
    last_week = date_list[i-1]
    first_week = date_list[i-52]
    
    this_week_data = (dateData.loc[this_week]).copy()
    last_week_data = (dateData.loc[last_week]).copy()
    
    liquid_list = ((last_week_data.sort_values(by=['total_volume'], ascending=False)).index)[:10]
    
    liquid_data = pd.DataFrame(index= date_list[i-52 : i], columns = list(liquid_list))
    
    for ticker in liquid_list:
        liquid_data[ticker] = tickerData.Returns.loc[ticker].loc[first_week:this_week]
        
    weights = getLongOnlyWeights(liquid_data)
    totalReturns = 0
    
    for ticker in liquid_list:
        totalReturns += weights[ticker]['weight'] * this_week_data.Returns.loc[ticker]
        
    liquid_LongOnly_returns.loc[this_week,'liquid_LongOnly_returns'] = totalReturns

In [94]:
marketcap_LongAndShort_returns = pd.DataFrame(index= date_list[104 : 156], columns = ['marketcap_LongAndShort_returns'])

for i in range(104,156,1):
    this_week = date_list[i]
    last_week = date_list[i-1]
    first_week = date_list[i-52]
    
    this_week_data = (dateData.loc[this_week]).copy()
    last_week_data = (dateData.loc[last_week]).copy()
    
    marketcap_list = ((last_week_data.sort_values(by=['market_cap'], ascending=False)).index)[:10]
    
    marketcap_data = pd.DataFrame(index= date_list[i-52 : i], columns = list(marketcap_list))
    
    for ticker in marketcap_list:
        marketcap_data[ticker] = tickerData.Returns.loc[ticker].loc[first_week:this_week]
        
    weights = getLongAndShortWeights(marketcap_data)
    totalReturns = 0
    
    for ticker in marketcap_list:
        totalReturns += weights[ticker]['weight'] * this_week_data.Returns.loc[ticker]
        
    marketcap_LongAndShort_returns.loc[this_week,'marketcap_LongAndShort_returns'] = totalReturns

In [95]:
liquid_LongAndShort_returns = pd.DataFrame(index= date_list[104 : 156], columns = ['liquid_LongAndShort_returns'])

for i in range(104,156,1):
    this_week = date_list[i]
    last_week = date_list[i-1]
    first_week = date_list[i-52]
    
    this_week_data = (dateData.loc[this_week]).copy()
    last_week_data = (dateData.loc[last_week]).copy()
    
    liquid_list = ((last_week_data.sort_values(by=['total_volume'], ascending=False)).index)[:10]
    
    liquid_data = pd.DataFrame(index= date_list[i-52 : i], columns = list(liquid_list))
    
    for ticker in liquid_list:
        liquid_data[ticker] = tickerData.Returns.loc[ticker].loc[first_week:this_week]
        
    weights = getLongAndShortWeights(liquid_data)
    totalReturns = 0
    
    for ticker in liquid_list:
        totalReturns += weights[ticker]['weight'] * this_week_data.Returns.loc[ticker]
        
    liquid_LongAndShort_returns.loc[this_week,'liquid_LongAndShort_returns'] = totalReturns

In [96]:
marketcap_Equal_returns = pd.DataFrame(index= date_list[104 : 156], columns = ['marketcap_Equal_returns'])

for i in range(104,156,1):
    this_week = date_list[i]
    last_week = date_list[i-1]
    first_week = date_list[i-52]
    
    this_week_data = (dateData.loc[this_week]).copy()
    last_week_data = (dateData.loc[last_week]).copy()
    
    marketcap_list = ((last_week_data.sort_values(by=['market_cap'], ascending=False)).index)[:10]
    
    marketcap_data = pd.DataFrame(index= date_list[i-52 : i], columns = list(marketcap_list))
    
    for ticker in marketcap_list:
        marketcap_data[ticker] = tickerData.Returns.loc[ticker].loc[first_week:this_week]
        
    totalReturns = 0
    
    for ticker in marketcap_list:
        totalReturns += 0.1 * this_week_data.Returns.loc[ticker]
        
    marketcap_Equal_returns.loc[this_week,'marketcap_Equal_returns'] = totalReturns

In [97]:
liquid_Equal_returns = pd.DataFrame(index= date_list[104 : 156], columns = ['liquid_Equal_returns'])

for i in range(104,156,1):
    this_week = date_list[i]
    last_week = date_list[i-1]
    first_week = date_list[i-52]
    
    this_week_data = (dateData.loc[this_week]).copy()
    last_week_data = (dateData.loc[last_week]).copy()
    
    liquid_list = ((last_week_data.sort_values(by=['total_volume'], ascending=False)).index)[:10]
    
    liquid_data = pd.DataFrame(index= date_list[i-52 : i], columns = list(liquid_list))
    
    for ticker in liquid_list:
        liquid_data[ticker] = tickerData.Returns.loc[ticker].loc[first_week:this_week]
        
    totalReturns = 0
    
    for ticker in liquid_list:
        totalReturns += 0.1 * this_week_data.Returns.loc[ticker]
        
    liquid_Equal_returns.loc[this_week,'liquid_Equal_returns'] = totalReturns

In [98]:
ALL_RETURNS = pd.concat([liquid_Equal_returns, marketcap_Equal_returns, liquid_LongAndShort_returns, marketcap_LongAndShort_returns, liquid_LongOnly_returns, marketcap_LongOnly_returns], axis=1)
ALL_RETURNS

,liquid_Equal_returns,marketcap_Equal_returns,liquid_LongAndShort_returns,marketcap_LongAndShort_returns,liquid_LongOnly_returns,marketcap_LongOnly_returns
date,,,,,,
2020-01-05,0.023307,0.016082,0.027773,0.050079,0.005951,0.007305
2020-01-12,0.10811,0.096104,0.178226,0.089706,0.089876,0.085619
2020-01-19,0.281107,0.211864,0.116095,0.162643,0.111683,0.142624
2020-01-26,-0.066996,-0.072309,-0.012785,-0.012904,-0.052277,-0.057064
2020-02-02,0.184672,0.168734,0.36024,0.158293,0.168485,0.105052
2020-02-09,0.118776,0.127385,-0.0263,0.013042,0.043623,0.080182
2020-02-16,0.035568,0.040145,-0.023827,-0.119664,-0.032363,-0.003151
2020-02-23,-0.088852,-0.08354,0.089324,-0.019971,-0.037499,-0.047872
2020-03-01,-0.174505,-0.148266,-0.170421,0.020437,-0.13174,-0.074653


Read data for market returns

In [99]:
CC100_Index = pd.read_csv("indexReturns-BITW100.csv")
CC100_Index['date'] = pd.to_datetime(CC100_Index['date']).dt.tz_localize(None)
CC100_Index = CC100_Index.sort_values('date').set_index('date')
CC100_Index = CC100_Index.truncate(before = TRAIN_START,after = TEST_END)
CC100_Index = CC100_Index.resample('W').last()
CC100_Return = CC100_Index.pct_change().fillna(axis = 0, method ='bfill').rename(columns = {"MarketIndex":"MarketReturn"})
CC100_Return = CC100_Return.iloc[1:].copy()
date_list = CC100_Return.index
CC100_Return.set_index(date_list, drop=True, inplace=True)

CC100_Return

,MarketReturn
date,
2018-01-07,0.407408
2018-01-14,-0.102655
2018-01-21,-0.156306
2018-01-28,-0.117886
2018-02-04,-0.202010
...,...
2020-11-29,-0.001030
2020-12-06,0.047348
2020-12-13,-0.013224


In [100]:
ALL_RETURNS['Market_Return'] = CC100_Return.loc['2020-01-05':]
cumulative_returns = (ALL_RETURNS + 1).cumprod()
cumulative_returns.iloc[-1]

liquid_Equal_returns               2.14039
marketcap_Equal_returns           2.615236
liquid_LongAndShort_returns       4.111672
marketcap_LongAndShort_returns    2.230988
liquid_LongOnly_returns           2.753164
marketcap_LongOnly_returns        2.295158
Market_Return                     3.340097
Name: 2020-12-27 00:00:00, dtype: object

In [101]:
cumulative_returns = cumulative_returns.astype(float)
ALL_RETURNS = ALL_RETURNS.astype(float)

In [102]:
ALL_RETURNS

,liquid_Equal_returns,marketcap_Equal_returns,liquid_LongAndShort_returns,marketcap_LongAndShort_returns,liquid_LongOnly_returns,marketcap_LongOnly_returns,Market_Return
date,,,,,,,
2020-01-05,0.023307,0.016082,0.027773,0.050079,0.005951,0.007305,0.009351
2020-01-12,0.108110,0.096104,0.178226,0.089706,0.089876,0.085619,0.093097
2020-01-19,0.281107,0.211864,0.116095,0.162643,0.111683,0.142624,0.082518
2020-01-26,-0.066996,-0.072309,-0.012785,-0.012904,-0.052277,-0.057064,-0.004803
2020-02-02,0.184672,0.168734,0.360240,0.158293,0.168485,0.105052,0.117997
2020-02-09,0.118776,0.127385,-0.026300,0.013042,0.043623,0.080182,0.085721
2020-02-16,0.035568,0.040145,-0.023827,-0.119664,-0.032363,-0.003151,-0.015518
2020-02-23,-0.088852,-0.083540,0.089324,-0.019971,-0.037499,-0.047872,0.017986
2020-03-01,-0.174505,-0.148266,-0.170421,0.020437,-0.131740,-0.074653,-0.162869


In [103]:
empirical_results = pd.DataFrame(columns=cumulative_returns.columns, index=['Return','Sharpe_ratio','Calmar_ratio','Tail_ratio'])

for column in ALL_RETURNS.columns:
    
    empirical_results.loc['Return'][column] =  cumulative_returns.loc['2020-12-27',column]
    empirical_results.loc['Sharpe_ratio'][column] = empyrical.sharpe_ratio(ALL_RETURNS.loc[:,column],risk_free=0,period = 'weekly')
    empirical_results.loc['Calmar_ratio'][column] = empyrical.calmar_ratio(ALL_RETURNS.loc[:,column],period = 'weekly')
    empirical_results.loc['Tail_ratio'][column] = empyrical.tail_ratio(ALL_RETURNS.loc[:,column])

In [104]:
empirical_results = (empirical_results.T)
print(empirical_results) 

                                  Return Sharpe_ratio Calmar_ratio Tail_ratio
liquid_Equal_returns             2.14039      1.35574      2.01608   1.513461
marketcap_Equal_returns         2.615236     1.640046     2.928353   1.679989
liquid_LongAndShort_returns     4.111672     1.709549     5.349758   1.903028
marketcap_LongAndShort_returns  2.230988     1.497194     1.586653   1.151935
liquid_LongOnly_returns         2.753164     1.491824     3.135088   1.368273
marketcap_LongOnly_returns      2.295158     1.394294     2.476767   1.463836
Market_Return                   3.340097     2.196022     4.848944   1.543315
